In [1]:
# Import
import pandas as pd
from finta import TA as ta
from pandas.tseries.offsets import DateOffset
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from pathlib import Path

# Setting these options will allow for reviewing more of the DataFrames
pd.set_option('display.max_rows', 2000)
pd.set_option('display.max_columns', 2000)
pd.set_option('display.width', 1000)

In [10]:
ticker1 = "BTC"
ticker2 = "USDT"
pair= "{}{}".format(ticker1,ticker2)
start="2019.1.1"
end = "2022.4.1"
filepath = "Resources/{}_{}_to_{}.csv".format(pair,start,end)

df = pd.read_csv(Path(filepath))
display(df.head())
display(df.tail())

,Date,Open,High,Low,Close,Volume
0,2019-01-01 00:00:00,3701.23,3713.00,3689.88,3700.31,686.367420
1,2019-01-01 01:00:00,3700.20,3702.73,3684.22,3689.69,613.539115
2,2019-01-01 02:00:00,3689.67,3695.95,3675.04,3690.00,895.302181
3,2019-01-01 03:00:00,3690.00,3699.77,3685.78,3693.13,796.714818
4,2019-01-01 04:00:00,3692.32,3720.00,3685.94,3692.71,1317.452909


,Date,Open,High,Low,Close,Volume
28402,2022-03-31 20:00:00,45581.98,45790.00,45519.03,45745.46,1550.11357
28403,2022-03-31 21:00:00,45745.45,45849.99,45647.01,45757.19,887.89890
28404,2022-03-31 22:00:00,45757.19,45820.34,45660.68,45787.85,1058.12591
28405,2022-03-31 23:00:00,45787.85,45787.86,45200.00,45510.34,2889.28168
28406,2022-04-01 00:00:00,45510.35,45644.86,45351.73,45541.50,1838.20071


In [3]:
def appendData(maindf, dataarray, namesarray=None):
    if namesarray==None:
        return maindf.join(pd.DataFrame(dataarray), how='outer')
    return maindf.join(pd.DataFrame(dataarray,columns=namesarray), how='outer')

In [4]:
### Oscillators ###
## RSI
df = appendData(df,ta.RSI(df))
## Sto-%K
df = appendData(df,ta.STOCH(df))
## CCI
df = appendData(df,ta.CCI(df))
## ADX
df = appendData(df,ta.ADX(df))
## DMI (Added to aid in interpreting ADX)
df = appendData(df,ta.DMI(df, 14))
## Awesome
df = appendData(df,ta.AO(df))
## Momentum
df = appendData(df,ta.MOM(df,10))
## MACD (We rename the undescriptive "SIGNAL" here)
df = appendData(df,ta.MACD(df)).rename(columns={"SIGNAL": "MACD SIGNAL"})
## Sto-RSI
df = appendData(df,ta.STOCHRSI(df))
## Williams %R
df = appendData(df,ta.WILLIAMS(df))
## Bull-Bear Power
df = appendData(df,ta.EBBP(df))
## Ultimate (FinTA does not name this column, so we must)
df = appendData(df,ta.UO(df),["UO"])
### Moving Averages ###
sma_ema_averages = [5, 10, 20, 30, 50, 100, 200]
## SMA, EMA
for i in sma_ema_averages:
  df = appendData(df,ta.SMA(df, i))
  df = appendData(df,ta.EMA(df, i))
## VWMA
df = appendData(df, ta.VAMA(df, 20))
## Hull
df = appendData(df,ta.HMA(df, 9))
# Ichimoku -- Base (Kijun) and Conversion (Tenkan) Only
df = appendData(df,ta.ICHIMOKU(df).drop(['senkou_span_a','SENKOU','CHIKOU'], axis=1))

In [5]:
# Use the pct_change function to generate the returns from "close"
df["actual_return"] = df["Close"].pct_change()

df["three_period_return"] = df["Close"].pct_change(periods=3)

df["five_period_return"] = df["Close"].pct_change(periods=5)
# Drop all NaN values from the DataFrame
df = df.dropna()

df.head()

,Date,Open,High,Low,Close,Volume,14 period RSI,14 period STOCH %K,20 period CCI,14 period ADX.,DI+,DI-,AO,MOM,MACD,MACD SIGNAL,14 period stochastic RSI.,14 Williams %R,Bull.,Bear.,UO,5 period SMA,5 period EMA,10 period SMA,10 period EMA,20 period SMA,20 period EMA,30 period SMA,30 period EMA,50 period SMA,50 period EMA,100 period SMA,100 period EMA,200 period SMA,200 period EMA,20 period VAMA,9 period HMA.,TENKAN,KIJUN,actual_return,three_period_return,five_period_return
199,2019-01-09 07:00:00,3986.44,4000.00,3985.02,3987.88,797.726342,54.817444,85.737374,41.204569,16.120448,19.642015,14.956418,15.416118,24.65,4.485321,3.255189,0.530570,-14.262626,19.241559,4.261559,53.945063,3987.504,3985.853744,3978.285,3982.099488,3980.5835,3978.616353,3974.350333,3974.991184,3975.3134,3962.066090,3909.3316,3925.339590,3839.34730,3887.502495,3985.351148,3992.654148,3952.500,3986.4,0.000314,-0.001532,0.001819
200,2019-01-09 08:00:00,3987.76,3991.78,3971.11,3982.54,1076.596718,52.489020,80.343434,6.305424,15.087222,18.239013,17.645026,14.158853,14.71,4.263661,3.456883,0.533353,-19.656566,10.767051,-9.902949,45.953590,3987.806,3984.749163,3979.756,3982.179581,3980.2690,3978.990034,3975.463000,3975.478205,3975.1486,3962.869247,3910.9998,3926.492976,3840.75845,3888.594449,3985.043136,3988.821704,3966.020,3986.4,-0.001339,-0.001364,0.000379
201,2019-01-09 09:00:00,3982.85,3996.98,3976.00,3978.00,930.436261,50.524117,75.757576,24.430281,14.414950,18.356370,16.384666,12.568265,22.87,3.679241,3.501355,0.533035,-24.242424,16.397472,-4.582528,44.321302,3984.606,3982.499442,3982.043,3981.419657,3977.8330,3978.895745,3976.542667,3975.640902,3974.6886,3963.462793,3912.4397,3927.531184,3842.20000,3889.620114,3980.363661,3983.713667,3969.130,3986.4,-0.001140,-0.002165,-0.004006
202,2019-01-09 10:00:00,3977.70,3987.07,3971.50,3982.77,1215.609054,52.534585,80.575758,19.081687,13.510125,17.045200,16.459732,10.267324,27.04,3.559947,3.513073,0.536520,-19.424242,6.174976,-9.395024,48.941642,3983.564,3982.589628,3984.747,3981.665174,3976.5310,3979.264722,3977.805667,3976.100844,3974.1900,3964.220164,3914.2729,3928.644218,3843.66385,3890.687111,3977.917927,3980.653037,3986.555,3986.4,0.001199,-0.001281,-0.001306
203,2019-01-09 11:00:00,3982.53,4006.81,3980.58,3986.44,1377.455574,54.080664,80.377613,96.017288,13.692523,21.134140,15.284036,11.540294,0.44,3.718677,3.554194,0.541618,-19.622387,25.122837,-1.107163,44.416976,3983.526,3983.873085,3984.791,3982.533324,3975.8290,3979.948081,3979.004000,3976.767887,3974.7786,3965.091779,3916.2858,3929.808369,3845.13040,3891.782276,3974.168231,3980.857333,3988.960,3986.4,0.000921,0.000979,-0.000048


In [6]:

investment_amount = 10000 * .05

# Initialize the new `Signal` column
df['signal'] = 0.0
# Generate signal to buy stock long
df.loc[(df['three_period_return'] >= ((investment_amount * 0.00001))), 'signal'] = 1
# Generate signal to sell stock short
#df.loc[(df['three_period_return'] < (investment_amount * 0.00001)), 'signal'] = 0

In [7]:
# Copy the new "signal" column to a new Series called `y`.
y = df['signal']
# set up X for ml 
X = df[["14 period RSI", "14 period STOCH %K", "20 period CCI", "14 period ADX.", "DI+", "DI-", "AO", "MOM", "MACD", "MACD SIGNAL", "14 period stochastic RSI.", "14 Williams %R", "Bull.", "Bear.", "UO", "5 period SMA", "5 period EMA", "10 period SMA", "10 period EMA", "20 period SMA", "20 period EMA", "30 period SMA", "30 period EMA", "50 period SMA", "50 period EMA", "100 period SMA", "100 period EMA", "200 period SMA", "200 period EMA", "20 period VAMA", "9 period HMA.", "TENKAN", "KIJUN"]].shift().dropna().copy()

In [8]:
# value counts for -1 and 1
y.value_counts()

0.0    20847
1.0     7361
Name: signal, dtype: int64

In [9]:
# Use the following code to select the start of the training period: `training_begin = X.index.min()`
training_begin = X.index.min()
print(training_begin)

# Use the following code to select the ending period for the training data: `training_end = X.index.min() + DateOffset(months=3)`
training_end = X.index.min() + DateOffset(months=3)
print(training_end)

# Generate the X_train and y_train DataFrames using loc to select the rows from `training_begin` up to `training_end`
# Hint: Use `loc[training_begin:training_end]` for X_train and y_train
X_train = X.loc[training_begin:training_end]
y_train = y.loc[training_begin:training_end]

# Generate the X_test and y_test DataFrames using loc to select from `training_end` to the last row in the DataFrame.
# Hint: Use `loc[training_end:]` for X_test and y_test
X_test = X.loc[training_end:]
y_test = y.loc[training_end:]

# Use StandardScaler to scale the X_train and X_test data.
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_scaler = scaler.fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

200


TypeError: unsupported operand type(s) for +: 'int' and 'DateOffset'

In [ ]:
model = SVC()
 
# Fit the model to the data using X_train_scaled and y_train
model = model.fit(X_train_scaled, y_train)

# Use the trained model to predict the trading signals for the training data.
training_signal_predictions = model.predict(X_train_scaled)

# Evaluate the model using a classification report
from sklearn.metrics import classification_report
training_report = classification_report(y_train, training_signal_predictions)
print(training_report)

In [ ]:
# Use the trained model to predict the trading signals for the testing data.
testing_signal_predictions = model.predict(X_test_scaled)

# Evaluate the model's ability to predict the trading signal for the testing data using a classification report
training_report = classification_report(y_test, testing_signal_predictions)
print(training_report)

In [ ]:
 # Create a new empty predictions DataFrame using code provided below.
predictions_df = pd.DataFrame(index=X_test.index)
predictions_df['signal'] = testing_signal_predictions
predictions_df['signal'].value_counts()
predictions_df["three_period_return"] = df["three_period_return"]


intial_investment = 10000
# Add in actual returns and calculate trading returns
predictions_df['actual_return'] = df['actual_return']
predictions_df['trading_algorithm_returns_3'] = predictions_df.loc[predictions_df["signal"]==1]["three_period_return"]
#predictions_df["trading_algorithm_returns_5"] = 
predictions_df["three_day_return"] = (1 + predictions_df["trading_algorithm_returns_3"]).cumprod() - 1
# we need to add trading algo cumulative returns in order to plot agaisnt actual returns and see how well our algo performed compared to actual returns

predictions_df[50:100]

In [ ]:
# Calculate and plot the cumulative returns for the `actual_returns` and the `trading_algorithm_returns`

(1 + predictions_df[['actual_return', 'trading_algo_returns_3_+1']]).cumprod().plot()